## Test the Backend Agent API

This notebook is used to test the FastAPI application running either locally or in a Docker container or in a containerized hosting service.

In [ ]:
import requests


api_url = "http://localhost:8000"   # FastAPI uvicorn URL with port 8000
# api_url = "http://localhost:80"     # Docker container URL since we exposed the port 80
# api_url = "https://chinook-backend-api.azurewebsites.net"  # Azure Web App URL
# api_url = "http://20.118.71.68:80"  # AKS URL

res = requests.get(f"{api_url}/health")
res.json()

{'status': '🤙'}

In [2]:
res.status_code

200

In [3]:
res.text

'{"status":"🤙"}'

In [4]:
res = requests.post(f"{api_url}/sql-invoke",
    json={
        "message": "Find albums released by artists who have more than 5 albums",
        "thread_id": "847c6285-8fc9-4560-a83f-4e6285809364"
    }
)
res

<Response [200]>

In [5]:
res.json()

{'type': 'ai',
 'content': "Here are some albums released by artists who have more than 5 albums:\n\n1. **Deep Purple**\n   - Come Taste The Band\n   - Deep Purple In Rock\n   - Fireball\n   - Knocking at Your Back Door: The Best Of Deep Purple in the 80's\n   - MK III The Final Concerts [Disc 1]\n   - Machine Head\n   - Purpendicular\n   - Slaves And Masters\n   - Stormbringer\n   - The Battle Rages On\n   - The Final Concerts (Disc 2)\n\n2. **Iron Maiden**\n   - A Matter of Life and Death\n   - A Real Dead One\n   - A Real Live One\n   - Brave New World\n   - Dance Of Death\n   - Fear Of The Dark\n   - Iron Maiden\n   - Killers\n   - Live After Death\n   - Live At Donington 1992 (Disc 1)\n   - Live At Donington 1992 (Disc 2)\n   - No Prayer For The Dying\n   - Piece Of Mind\n   - Powerslave\n   - Rock In Rio [CD1]\n   - Rock In Rio [CD2]\n   - Seventh Son of a Seventh Son\n   - Somewhere in Time\n   - The Number of The Beast\n   - The X Factor\n   - Virtual XI\n\n3. **Led Zeppelin**\

In [12]:
print(res.json()['content'])

Here are some albums released by artists who have more than 5 albums:

1. **Deep Purple**
   - Come Taste The Band
   - Deep Purple In Rock
   - Fireball
   - Knocking at Your Back Door: The Best Of Deep Purple in the 80's
   - MK III The Final Concerts [Disc 1]
   - Machine Head
   - Purpendicular
   - Slaves And Masters
   - Stormbringer
   - The Battle Rages On
   - The Final Concerts (Disc 2)

2. **Iron Maiden**
   - A Matter of Life and Death
   - A Real Dead One
   - A Real Live One
   - Brave New World
   - Dance Of Death
   - Fear Of The Dark
   - Iron Maiden
   - Killers
   - Live After Death
   - Live At Donington 1992 (Disc 1)
   - Live At Donington 1992 (Disc 2)
   - No Prayer For The Dying
   - Piece Of Mind
   - Powerslave
   - Rock In Rio [CD1]
   - Rock In Rio [CD2]
   - Seventh Son of a Seventh Son
   - Somewhere in Time
   - The Number of The Beast
   - The X Factor
   - Virtual XI

3. **Led Zeppelin**
   - BBC Sessions [Disc 1] [Live]
   - BBC Sessions [Disc 2] [Live

## Invoke Endpoint Test

In [13]:
def invoke_sql_query(message, thread_id):
    try:        
        res = requests.post(f"{api_url}/sql-invoke",
            json={
                "message": message,
                "thread_id": thread_id
            }
        )
        print(res.json()["content"])
    except Exception as e:
        print(e)
        

In [14]:
import uuid
thread_id = str(uuid.uuid4())
invoke_sql_query("Can you find the total revenue generated from all sales?", thread_id)

The total revenue generated from all sales is 2328.6.


In [15]:
invoke_sql_query("Who is the highest-paying customer?", thread_id)

The highest-paying customer is Helena Holý, who has spent a total of 49.62.


In [16]:
invoke_sql_query("What is the most popular track by number of purchases?", thread_id)

The most popular track by number of purchases is "The Trooper," with 5 purchases.


In [17]:
invoke_sql_query("Who are the top 5 employees who have made the most sales?", thread_id)

The top 3 employees who have made the most sales are:

1. Jane Peacock with 146 sales.
2. Margaret Park with 140 sales.
3. Steve Johnson with 126 sales.

It seems there are only three employees in the data available.


## Stream Endpoint Test

In [ ]:
# import httpx  
# import json
# import plotly.io as pio
# import plotly.graph_objects as go
# from IPython.display import Markdown, display

# async def sql_stream(message: str, thread_id: str):
#     url = f"{api_url}/sql-stream"

#     payload = {  
#         "message": message,  
#         "thread_id": thread_id,  
#         "stream_tokens": True  
#     }  

#     headers = {'Content-Type': 'application/json'}

#     async with httpx.AsyncClient() as client:
#         async with client.stream("POST", url, json=payload, headers=headers, timeout=None) as response:
#             if response.status_code == 200:  
#                 async for line in response.aiter_lines():
#                     if line:
#                         display(Markdown(line))  # Stream and display text immediately
#                         # print(line)

#             else:  
#                 print(f"Failed to get data: {response.status_code}")


In [ ]:
# When plotly JSON is received, it will be displayed as a chart

import httpx  
import json
import plotly.io as pio
import plotly.graph_objects as go
from IPython.display import Markdown, display

async def sql_stream(message: str, thread_id: str):
    url = f"{api_url}/sql-stream"

    payload = {  
        "message": message,  
        "thread_id": thread_id,  
        "stream_tokens": True  
    }  

    headers = {'Content-Type': 'application/json'}
    json_buffer = ""  # Store potential JSON content
    found_json = False  # Track if JSON block has started

    async with httpx.AsyncClient() as client:
        async with client.stream("POST", url, json=payload, headers=headers, timeout=None) as response:
            if response.status_code == 200:  
                async for line in response.aiter_lines():
                    if line:
                        # print(f"Received token: {line}")  # Debugging: Print each received token
                        

                        # If "plotly_json:" appears, ignore it and extract JSON only
                        if "plotly_json:" in line:
                            found_json = True
                            json_buffer = line.split("plotly_json:", 1)[-1].strip()  # Extract JSON content
                        elif found_json:
                            json_buffer += line.strip()  # Continue collecting JSON if already found
                        else:
                            display(Markdown(line))  # Stream and display text immediately
                            # print(line)

            else:  
                print(f"Failed to get data: {response.status_code}")

    # If JSON was found, try to parse and display it
    if found_json and json_buffer:
        try:
            # print(f"Final extracted JSON:\n{json_buffer}")  # Debugging: Print the extracted JSON
            plotly_json = json.loads(json_buffer)  # Convert string to dictionary
            fig = go.Figure(pio.from_json(json.dumps(plotly_json)))  # Create Plotly figure
            display(fig)  # Display the chart
        except Exception as e:
            print("Error rendering Plotly chart:", str(e))


In [3]:
import uuid
thread_id = str(uuid.uuid4())
await sql_stream("Can you tell me the names of popular albums in the database?", thread_id)

Here are some popular album titles from the database:

1. Koyaanisqatsi (Soundtrack from the Motion Picture)

2. Mozart: Chamber Music

3. Monteverdi: L'Orfeo

4. Schubert: The Late String Quartets & String Quintet (3 CDs)

...

7. Great Recordings of the Century - Shubert: Schwanengesang, 4 Lieder

8. Liszt - 12 Études D'Execution Transcendante

9. Great Recordings of the Century: Paganini's 24 Caprices

10. Nielsen: The Six Symphonies

If you need more information or additional titles, feel free to ask!

In [4]:
await sql_stream("Find albums released by artists who have more than 5 albums", thread_id)

Here are some albums released by artists who have more than 5 albums:

**Deep Purple:**

1. Come Taste The Band

2. Deep Purple In Rock

3. Fireball

...

1. ...And Justice For All

2. Black Album

3. Garage Inc. (Disc 1)

4. Garage Inc. (Disc 2)

If you need more information or additional titles, feel free to ask!